In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q git+https://github.com/openai/whisper.git
!apt update && apt install -y ffmpeg


In [ ]:
!pip install OpenCC

In [ ]:
start_index = 0
end_index = 3
folder_path = '/content/drive/My Drive/Ephesians_Book/'
audio_file_prefix = 'part'
text_file_prefix = audio_file_prefix

In [ ]:
from opencc import OpenCC
import whisper

In [ ]:
cc = OpenCC('t2s')

In [ ]:


# Load the model (you can use "small", "medium", or "large" — large = best accuracy)
model = whisper.load_model("small")  # medium is good for Chinese & faster than large

In [ ]:
def file_name_builder(folder_path, file_prefix, file_extension, index):
  return f"{folder_path}{file_prefix}_{index:03d}.{file_extension}"

In [ ]:
%ls '/content/drive/My Drive/'

In [ ]:
for i in range(start_index, end_index):
  audio_file_name = file_name_builder(f"{folder_path}audio_files_experiment/",
                                      audio_file_prefix, "m4a", i)
  print(f"processing file {audio_file_name}")

  result = model.transcribe(audio_file_name, language="zh")
  # convert any traditional Chinese to simplified Chinese
  result["text"] = cc.convert(result["text"])

  text_file_name = file_name_builder(f"{folder_path}text_files_experiment/",
                                    text_file_prefix, "txt", i)
  with open(text_file_name, "w", encoding="utf-8") as f:
    f.write(result["text"])